## Importing

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets

In [34]:
reddit = pd.read_csv(r"C:\Users\User\Downloads\Reddit_Data.csv")
reddit.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


## Cleaning the text

### Converting to lower case

In [35]:
reddit['clean_comment'] = reddit['clean_comment'].str.lower()
reddit.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


### Removing stop words

In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

tokenizer=ToktokTokenizer() 
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [37]:
reddit['clean_comment'] = reddit.clean_comment.apply(remove_stopwords)

In [38]:
reddit['clean_comment'].head()

0    family mormon never tried explain still stare ...
1    buddhism much lot compatible christianity espe...
2    seriously say thing first get complex explain ...
3    learned want teach different focus goal wrappi...
4    benefit may want read living buddha living chr...
Name: clean_comment, dtype: object

### Removing URLs

In [39]:
import re

def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
reddit['clean_comment'] = reddit['clean_comment'].apply(lambda x: cleaning_URLs(x))
reddit['clean_comment'].head()

0    family mormon never tried explain still stare ...
1    buddhism much lot compatible christianity espe...
2    seriously say thing first get complex explain ...
3    learned want teach different focus goal wrappi...
4    benefit may want read living buddha living chr...
Name: clean_comment, dtype: object

### Removing Numbers

In [40]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
reddit['clean_comment'] = reddit['clean_comment'].apply(lambda x: cleaning_numbers(x))
reddit['clean_comment'].head()

0    family mormon never tried explain still stare ...
1    buddhism much lot compatible christianity espe...
2    seriously say thing first get complex explain ...
3    learned want teach different focus goal wrappi...
4    benefit may want read living buddha living chr...
Name: clean_comment, dtype: object

In [41]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [42]:
reddit['clean_comment'] = np.vectorize(remove_pattern)(reddit['clean_comment'], "@[\w]*")
reddit.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal wrappi...,0
4,benefit may want read living buddha living chr...,1


In [43]:
# remove special characters, numbers and punctuations
reddit['clean_comment'] = reddit['clean_comment'].str.replace(r'[^a-zA-Z#]', ' ')
reddit.head()

C:\Users\User\AppData\Local\Temp\ipykernel_14812\4282037118.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reddit['clean_comment'] = reddit['clean_comment'].str.replace(r'[^a-zA-Z#]', ' ')


,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal wrappi...,0
4,benefit may want read living buddha living chr...,1


In [44]:
# remove short words, since they are not going to be giving any meaning like u, ur etc.,

reddit['clean_comment'] = reddit['clean_comment'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
reddit.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much compatible christianity especial...,1
2,seriously thing first complex explain normal p...,-1
3,learned want teach different focus goal wrappi...,0
4,benefit want read living buddha living christ ...,1


In [45]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 3

In [46]:
### Defining the feature processing

TEXT = torchtext.legacy.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)

In [47]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path='Reddit_Data.csv', format='csv',
    skip_header=True, fields=fields)

In [48]:
import random
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 29799
Num Test: 7450


In [49]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 25329
Num Validation: 4470


In [50]:
print(vars(train_data.examples[0]))

{'TEXT_COLUMN_NAME': [' ', 'there', 'any', 'explanation', 'for', 'why', 'all', 'the', 'giants', 'are', 'dead', 'jotunheim', 'seems', 'there', 'was', 'battle', 'there', 'blood', 'the', 'mountains', 'maybe', 'there', 'was', 'fight', 'about', 'what', 'with', 'loki', 'and', 'that', 'how', 'they', 'died', 'and', 'laufey', 'got', 'mortally', 'wounded'], 'LABEL_COLUMN_NAME': '-1'}


In [51]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 3


In [52]:
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1, '-1': 2})


In [53]:
train_loader, valid_loader, test_loader = \
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

In [54]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([594, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([2, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([1, 128])
Target vector size: torch.Size([128])


In [55]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [56]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [57]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [58]:
import time
import torch.nn.functional as F
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/198 | Loss: 1.0990
Epoch: 001/015 | Batch 050/198 | Loss: 1.0466
Epoch: 001/015 | Batch 100/198 | Loss: 1.0545
Epoch: 001/015 | Batch 150/198 | Loss: 1.0481
training accuracy: 42.50%
valid accuracy: 43.11%
Time elapsed: 27.10 min
Epoch: 002/015 | Batch 000/198 | Loss: 1.0672
Epoch: 002/015 | Batch 050/198 | Loss: 1.0135
Epoch: 002/015 | Batch 100/198 | Loss: 0.9781
Epoch: 002/015 | Batch 150/198 | Loss: 0.6725
training accuracy: 78.54%
valid accuracy: 61.25%
Time elapsed: 37.52 min
Epoch: 003/015 | Batch 000/198 | Loss: 0.6198
Epoch: 003/015 | Batch 050/198 | Loss: 0.3515
Epoch: 003/015 | Batch 100/198 | Loss: 0.5450
Epoch: 003/015 | Batch 150/198 | Loss: 0.3892
training accuracy: 90.70%
valid accuracy: 78.48%
Time elapsed: 43.53 min
Epoch: 004/015 | Batch 000/198 | Loss: 0.2666
Epoch: 004/015 | Batch 050/198 | Loss: 0.2228
Epoch: 004/015 | Batch 100/198 | Loss: 0.2836
Epoch: 004/015 | Batch 150/198 | Loss: 0.2245
training accuracy: 94.22%
valid accuracy: 82.

In [59]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.992868959903717

In [60]:
print('Probability negative:')
1-predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability negative:


0.993397630751133